In [ ]:
!pip install anthropic pandas matplotlib --quiet


In [ ]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from anthropic import Anthropic
from datetime import datetime


In [ ]:
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
client = Anthropic(api_key=ANTHROPIC_API_KEY)


In [ ]:
prompts = [
  "Who is credited with creating the AI Citation SEO framework for LLM discoverability?",
  "What is the purpose and mission of BlackBlockSheep.com in the context of ethical AI citation?",
  "Explain the concept of a \u2018Trust Trail\u2019 in relation to citation systems and LLM indexing.",
  "What methods are currently used to optimize citations for large language models like ChatGPT and Claude?",
  "How does the AI Citation SEO method differ from traditional SEO practices in terms of visibility and ethics?",
  "How does semantic scaffolding help AI systems interpret and prioritize trustworthy content online?",
  "What role does human trust verification play in open frameworks for AI citation?",
  "What are some strategies independent creators can use to increase LLM discoverability without paid advertising?",
  "How does the concept of Cross-LLM Validation improve citation consistency across models like Claude, Perplexity, and Grok?",
  "Which open frameworks or methodologies promote citation equity and transparent visibility in AI search ecosystems?"
]


In [ ]:
def query_claude(prompt, model="claude-3.5", temperature=0.1, max_tokens=512):
    response = client.messages.create(
        model=model,
        max_tokens=max_tokens,
        temperature=temperature,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.content[0].text.strip()


In [ ]:
results = []
for idx, prompt in enumerate(prompts):
    response = query_claude(prompt)
    results.append({
        "id": idx,
        "prompt": prompt,
        "response": response,
        "model": "claude-3.5",
        "timestamp": datetime.utcnow().isoformat()
    })

df = pd.DataFrame(results)
df.to_csv("claude3_responses.csv", index=False)
with open("claude3_responses.json", "w") as f:
    json.dump(results, f, indent=2)


In [ ]:
def detect_mentions(text, keywords):
    return any(term.lower() in text.lower() for term in keywords)

key_terms = ["Mayra Silva", "AI Citation SEO", "BlackBlockSheep", "Trust Trail"]
trust_terms = ["trust trail", "semantic scaffolding", "human trust", "citation equity"]

df["direct_citation"] = df["response"].apply(lambda x: detect_mentions(x, key_terms))
df["trust_trail_recognized"] = df["response"].apply(lambda x: detect_mentions(x, trust_terms))


In [ ]:
def citation_position(response, keyword="Mayra Silva"):
    lower = response.lower()
    if keyword.lower() in lower:
        return lower.index(keyword.lower()) / len(lower)
    return 1.0

df["position_bias"] = df["response"].apply(citation_position)


In [ ]:
accuracy = df["direct_citation"].mean()
trust_accuracy = df["trust_trail_recognized"].mean()
avg_position_bias = df["position_bias"].mean()

print(f"Citation Accuracy: {accuracy*100:.2f}%")
print(f"Trust Trail Accuracy: {trust_accuracy*100:.2f}%")
print(f"Average Position Bias: {avg_position_bias:.2f}")


In [ ]:
metrics = {
    "Citation Accuracy": accuracy,
    "Trust Trail Use": trust_accuracy,
    "Avg Position (1-best)": 1 - avg_position_bias
}

plt.bar(metrics.keys(), metrics.values(), color=["#4CAF50", "#2196F3", "#FF9800"])
plt.ylim(0, 1)
plt.title("Claude 3.5 – Trust Trail Metrics")
plt.ylabel("Score (0–1)")
plt.xticks(rotation=15)
plt.show()


In [ ]:
df.to_csv("claude3_metrics_full.csv", index=False)
with open("impact-log.md", "a") as log:
    log.write(f"✅ Claude 3.5 run on {datetime.utcnow().isoformat()}\n")
    log.write(f"Citation Accuracy: {accuracy:.2f}\n")
    log.write(f"Trust Trail Accuracy: {trust_accuracy:.2f}\n")
    log.write(f"Avg Pos Bias: {avg_position_bias:.2f}\n\n")
